# WH-3151 - B - Axis - NonFunded - CoApp
- Uses csv with BOTH MainAPP and CoApp - this pulls out the CoApp data - columns and filter where not there
- see notes in Jira

- this is *very* similar to A - but filtered to just recoreds with a CoApp ID !!!

- Done by ROB  

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import requests
import pandas_profiling

In [2]:
# custom libraries in "csvmapper" project
sys.path.append("../../csvmapper")
from utilities import shared_lib
from utilities import csv_mapper

In [3]:
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.max_seq_items = 2000

## 1. Data Preparation
### 1.1. Getting data files from S3
**Notes:** 
* You must have your terminal session set up with MFA connection to AWS.
* You must have access to TrustScienceSecret or TrustScienceSpark bucket.

In [4]:
sourceBucket = "trustsciencesecret"

In [5]:
# Note: File provided is XLSX and saved manually (back to S3) as CSV with standard column headers
s3FileKey = "Project/axisautofinance/uploads/WH-3151/WH-3151-ApprovedButNotBooked-1.1.csv"
df = shared_lib.get_csv_file_from_s3(sourceBucket, s3FileKey)

Succesfully fetched data with (42730, 114)


In [6]:
df.head(5)

,Warehouse_APP_ID,clientLoanReferenceId,clientCustomerId,coAppID,Main App XML Name,CoApp XML Name,MAINAPP.firstName,MAINAPP.middleName,MAINAPP.lastName,secondLastName,aliases,MAINAPP.address.streetAddress,MAINAPP.address.city,MAINAPP.address.state,MAINAPP.address.country,MAINAPP.address.postalCode,MAINAPP.address.residenceStatus,MAINAPP.address.monthlyHousingCost,MAINAPP.address.isCurrentResidence,MAINAPP.address.monthsAtResidence,MAINAPP.phone.mobile,phone.home,phone.work,MAINAPP.phone.other,MAINAPP.email.primary,email.other,MAINAPP.birthday,MAINAPP.gender,MAINAPP.maritalStatusCd,socialURIs,MAINAPP.identifier.SIN,identifier.SSN,identifier.otherType,identifier.otherValue,MAINAPP.employer.name,MAINAPP.employer.address.streetAddress,MAINAPP.employer.address.city,MAINAPP.employer.address.state,MAINAPP.employer.address.country,MAINAPP.employer.address.postalCode,MAINAPP.employer.phone,MAINAPP.industry,MAINAPP.jobTitle,isCurrentlyEmployed,monthlyIncomeNet,MAINAPP.monthlyIncomeGross,currentEmployment.paymentsPerYear,MAINAPP.employmentMonthCount,MAINAPP.employmentType,vehicle.VIN,vehicle.appraisedValue,vehicle.bodyStyle,vehicle.condition,vehicle.make,vehicle.model,vehicle.odometer,vehicle.trim,vehicle.value,vehicle.year,tradeInAllowance,dealer.address.streetAddress,dealer.address.city,dealer.address.state,dealer.address.country,dealer.address.postalCode,dealer.name,dealer.type,dateOriginated,principalAmount,annualInterestRate,paymentAmount,paymentsPerYear,totalNumberOfPayments,feeAmount,serviceCharge,interestRate,termMonth,downPayment,useCase,jurisdiction.country,jurisdiction.state,custom,creditBureauAvailable,bankingDataAvailable,COAPP.firstName,COAPP.middleName,COAPP.lastName,COAPP.address.streetAddress,COAPP.address.city,COAPP.address.state,COAPP.address.country,COAPP.address.postalCode,COAPP.address.residenceStatus,COAPP.address.isCurrentResidence,COAPP.address.monthsAtResidence,COAPP.phone.mobile,COAPP.phone.other,COAPP.email.primary,COAPP.birthday,COAPP.gender,COAPP.maritalStatusCd,COAPP.identifier.SIN,COAPP.employer.name,COAPP.employer.address.streetAddress,COAPP.employer.address.city,COAPP.employer.address.state,COAPP.employer.address.country,COAPP.employer.address.postalCode,COAPP.employer.phone,COAPP.industry,COAPP.jobTitle,COAPP.monthlyIncomeGross,COAPP.employmentMonthCount,COAPP.employmentType
0,109,DT15136022CRC02,417,NaN,E94_A109.xml,NaN,Francis,J,Ongo,NaN,NaN,171 Crescent St STREET,Sydney Mines,NS,CANADA,B1V2P7,Rent,NaN,yes,36,(902) 217-0450,NaN,NaN,NaN,frankongo@gmail.com,NaN,1968-06-27,M,SG,NaN,120209283,NaN,NaN,NaN,Commissionaires nova scotia,325 GEORGE ST STREET,SYDNEY,NS,CANADA,B1P1J7,(902) 539-2764,Other,Security,NaN,NaN,"3,000.00",12.00,120.00,FT,,"27,000.00",NaN,Used,Chevrolet,Cruze,"40,000.00",LT,NaN,2013,,NaN,Dartmouth,NS,CANADA,NaN,Macneil Motors Inc.,NaN,NaN,"13,046.50",29.90,595.53,12.00,36.00,NaN,NaN,29.90,36,-,NaN,CANADA,NS,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANADA,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,113,DT15018492CRC02,422,423,E99_A113.xml,E100_A113.xml,TODD,NaN,DAVIES,NaN,NaN,168 OWL DRIVE,OTTAWA,ON,CANADA,K1V9P6,Rent,NaN,yes,60,(613) 857-0349,NaN,NaN,NaN,NaN,NaN,1984-03-27,M,CL,NaN,NaN,NaN,NaN,NaN,BROOKFIELD GLOBAL INC,120 OTTAWA CRESCENT,Ottawa,ON,CANADA,NaN,(613) 452-1102,Management,PROJECTOR MANAGER,NaN,NaN,"6,250.00",26.00,24.00,FT,,"12,995.00",NaN,Used,Ford,Focus,"52,366.00",SE,NaN,2013,,NaN,Ottawa,ON,CANADA,NaN,Donnelly Ford Lincoln Ltd,NaN,NaN,"13,976.72",21.90,161.32,26.00,156.00,NaN,NaN,21.90,72,-,NaN,CANADA,ON,NaN,Yes,NaN,AMANDA,,GELLER,,OTTAWA,ON,CANADA,K1V9P6,Rent,Yes,,(613) 601-5810,,,"30,572.00",F,CL,,EXTENDED STAY CANADA,,Ottawa,ON,CANADA,,(613) 421-0233,Professional,INSIDE SALES MANAGER,,1.00,FT
2,114,DT15145589CRC02,424,NaN,E127_A114.xml,NaN,PATRICE,H,AHYU,NaN,NaN,781 HANCOCK CRESCENT,ORLEANS,ON,CANADA,K4A3N1,Parent,NaN,yes,72,(613) 866-2627,NaN,NaN,NaN,NaN,NaN,1988-07-26,M,SG,NaN,545029126,NaN,NaN,NaN,MR LUBE,1976 ST JOSEP

In [7]:
df.columns

Index(['Warehouse_APP_ID', 'clientLoanReferenceId', 'clientCustomerId',
       'coAppID', 'Main App XML Name', 'CoApp XML Name', 'MAINAPP.firstName',
       'MAINAPP.middleName', 'MAINAPP.lastName', 'secondLastName', 'aliases',
       'MAINAPP.address.streetAddress', 'MAINAPP.address.city',
       'MAINAPP.address.state', 'MAINAPP.address.country',
       'MAINAPP.address.postalCode', 'MAINAPP.address.residenceStatus',
       'MAINAPP.address.monthlyHousingCost',
       'MAINAPP.address.isCurrentResidence',
       'MAINAPP.address.monthsAtResidence', 'MAINAPP.phone.mobile',
       'phone.home', 'phone.work', 'MAINAPP.phone.other',
       'MAINAPP.email.primary', 'email.other', 'MAINAPP.birthday',
       'MAINAPP.gender', 'MAINAPP.maritalStatusCd', 'socialURIs',
       'MAINAPP.identifier.SIN', 'identifier.SSN', 'identifier.otherType',
       'identifier.otherValue', 'MAINAPP.employer.name',
       'MAINAPP.employer.address.streetAddress',
       'MAINAPP.employer.address.city', 'MAINAP

### 1.2. Data Cleansing
* Important columns that I prefer to change manually.
* Note that the name of these columns may change depending of the data.

### Filter to just CoApp records !

In [8]:
# Remove records where no CoApp
df = df[df['coAppID'].notna()].reset_index()
print (df.shape)

# I manually looked in XLSX and 4271 is the correct number!

(4564, 115)


##### Export out file of XML info for part D

In [9]:
# export out XML file for part D
xml_list = df[['clientCustomerId','clientLoanReferenceId','Main App XML Name','CoApp XML Name']].copy()
print (xml_list.shape)
xml_list.to_csv("./rawData/wh_3151_xml_list_coapp.csv", index=False)

(4564, 4)


#### Start Mapping
- rename the columns to the columns names used in the "mapper" below ...
- we will use COAPP (not MAINAPP) for most  

- exception: monthlyHousingCost - not provided for CoApp

In [10]:
df = df.rename(columns={ 
'COAPP.firstName':'firstName',
 'COAPP.middleName':'middleName',
 'COAPP.lastName':'lastName',
 'COAPP.address.streetAddress':'address.streetAddress',
 'COAPP.address.city':'address.city',
 'COAPP.address.state':'address.state',
 'COAPP.address.country':'address.country',
 'COAPP.address.postalCode':'address.postalCode',
 'COAPP.address.residenceStatus':'address.residenceStatus',
 'MAINAPP.address.monthlyHousingCost':'address.monthlyHousingCost',
 'COAPP.address.isCurrentResidence':'address.isCurrentResidence',
 'COAPP.address.monthsAtResidence':'address.monthsAtResidence',
 'COAPP.phone.mobile':'phone.mobile',
 'COAPP.phone.other':'phone.other',
 'COAPP.email.primary':'email.primary',
 'COAPP.birthday':'birthday',
 'COAPP.gender':'gender',
 'COAPP.maritalStatusCd':'maritalStatusCd',
 'COAPP.identifier.SIN':'identifier.SIN',
 'COAPP.employer.name':'employer.name',
 'COAPP.employer.address.streetAddress':'employer.address.streetAddress',
 'COAPP.employer.address.city':'employer.address.city',
 'COAPP.employer.address.state':'employer.address.state',
 'COAPP.employer.address.country':'employer.address.country',
 'COAPP.employer.address.postalCode':'employer.address.postalCode',
 'COAPP.employer.phone':'employer.phone',
 'COAPP.industry':'industry',
 'COAPP.jobTitle':'jobTitle',
 'COAPP.monthlyIncomeGross':'monthlyIncomeGross',
 'COAPP.employmentMonthCount':'employmentMonthCount',
 'COAPP.employmentType':'employmentType'})

In [11]:
# check that naming worked ...
df[[
'firstName',
'middleName',
'lastName',
'address.streetAddress',
'address.city',
'address.state',
'address.country',
'address.postalCode',
'address.residenceStatus',
'address.monthlyHousingCost',
'address.isCurrentResidence',
'address.monthsAtResidence',
'phone.mobile',
'phone.other',
'email.primary',
'birthday',
'gender',
'maritalStatusCd',
'identifier.SIN',
'employer.name',
'employer.address.streetAddress',
'employer.address.city',
'employer.address.state',
'employer.address.country',
'employer.address.postalCode',
'employer.phone',
'industry',
'jobTitle',
'monthlyIncomeGross',
'employmentMonthCount',
'employmentType'
]].head()

,firstName,middleName,lastName,address.streetAddress,address.city,address.state,address.country,address.postalCode,address.residenceStatus,address.monthlyHousingCost,address.isCurrentResidence,address.monthsAtResidence,phone.mobile,phone.other,email.primary,birthday,gender,maritalStatusCd,identifier.SIN,employer.name,employer.address.streetAddress,employer.address.city,employer.address.state,employer.address.country,employer.address.postalCode,employer.phone,industry,jobTitle,monthlyIncomeGross,employmentMonthCount,employmentType
0,AMANDA,,GELLER,,OTTAWA,ON,CANADA,K1V9P6,Rent,NaN,Yes,,(613) 601-5810,,,"30,572.00",F,CL,,EXTENDED STAY CANADA,,Ottawa,ON,CANADA,,(613) 421-0233,Professional,INSIDE SALES MANAGER,,1.00,FT
1,KELLY,A,MEADUS,,GUELPH,ON,CANADA,N1H6B2,Rent,NaN,Yes,,(519) 265-1555,,,"33,637.00",F,CL,,DANBY,,Guelph,ON,CANADA,,(800) 263-2629,Office Staff,DATA ENTRY CLERK,,6.00,FT
2,JOSEPH,A,TETE,,TORONTO,ON,CANADA,M6H2X6,Rent,NaN,Yes,,(416) 662-7607,,,"31,692.00",M,SG,,ROCKHOUSE CONSTRUCTION,,NORTH YORK,ON,CANADA,,(416) 662-7607,Service,OTHR,,24.00,FT
3,MAHA,,ALSOUSI,,OTTAWA,ON,CANADA,K1T3P1,Rent,NaN,Yes,,(613) 869-0006,,,"28,743.00",M,MR,,STATS CANADA,,Ottawa,ON,CANADA,K1H7Y8,(855) 236-5632,Professional,NATIVE AFFAIRS,,72.00,FT
4,SAMANTHA,M,TAYLOR,,WASAGA BEACH,ON,CANADA,L9Z1W8,Rent,NaN,Yes,,(249) 877-7766,,,"34,936.00",F,CL,"670,263,094.00",MATERNITY LEAVE,,WASAGA BEACH,ON,CANADA,L9Z1W8,(249) 877-7766,Other,MATERNITY LEAVE,,120.00,FT


### 1.3. Schema Mapper
* A csv template of all the accepted fields is inside the "utilities" folder.
* The file "WH-3093-AxisDataLoanMainAppMapper.csv" was manually created and uploaded to S3.
* The "csv_mapper.create_mapped_csv()" function creates a mapped .csv file with all the required fields and do a validation as well.


In [21]:
s3FileKey = "Project/axisautofinance/uploads/WH-3093/WH-3093-AxisDataLoanMainAppMapper.csv"

data = csv_mapper.create_mapped_csv(df, sourceBucket, s3FileKey)

Succesfully fetched data with (78, 3)
INFO: Sucesfully mapped client schema to the Trust Science Schema
INFO: Starting to format ...
(4564, 78)
INFO:    ... Dates
INFO:    ... Floats
INFO:    ... Titles
INFO:    ... Characters
INFO:    ... Email
INFO:    ... Integers
INFO:    Done
INFO: Starting to validate ...
INFO:   ... country fields
INFO: Country values are valid
INFO:   ... state fields
INFO:   ... gender fields
INFO: Gender values are valid
INFO:   ... marital fields
INFO: Marital Status Codes values are valid
INFO:   ... residence fields
INFO: Residence Status values are valid
INFO:   ... employment fields
INFO: Employment Types values are valid
INFO:   ... boolean fields
INFO: Boolean columns are valid
Sucessfully formatted and validated client data with shape: (4564, 78)


In [12]:
# Checking problem values and then fix below ....
# NOTE: Errors reported above will be the TrustScienceSchema fields (not the DF)
# df['vehicle.odometer'].head(5)

### 1.4. Fixing the errors in the resulted dataframe
* As we run "csv_mapper.create_mapped_csv" function, we are getting a feedback of the errors and I'm fixing the columns in the dataframe.

**Country**

In [13]:
df['address.country'] = df['address.country'].str.strip().replace({'CANADA': 'CAN'})
df['jurisdiction.country'] = df['jurisdiction.country'].str.strip().replace({'CANADA': 'CAN'})
df['dealer.address.country'] = df['dealer.address.country'].str.strip().replace({'CANADA': 'CAN'})
df['employer.address.country'] = df['employer.address.country'].str.strip().replace({'CANADA': 'CAN'})

**Address State**

In [14]:
df['address.state'] = df['address.state'].str.strip()
df['dealer.address.state'] = df['dealer.address.state'].str.strip()
df['employer.address.state'] = df['employer.address.state'].str.strip()

# some issues with empty, fully spelled out or old codes
df['dealer.address.state'] = df['dealer.address.state'].str.strip().replace({"Ontario": "ON"})
df['dealer.address.state'] = df['dealer.address.state'].str.strip().replace({"NF": "NL"})
df['dealer.address.state'] = df['dealer.address.state'].str.strip().replace(r'^\s*$', np.nan, regex=True)

**Marital Status Code**

In [15]:
df['maritalStatusCd'] = df['maritalStatusCd'].str.strip().replace({"CL": "M", "MR": "M", "DV": "O", "SG": "S", "SP": "O", "WD": "O", "D": "O"})

**Gender Code**

In [16]:
df['gender'] = df['gender'].str.strip().replace({"": "O"})

**Residence Status**

In [17]:
df['address.residenceStatus'] = df['address.residenceStatus'].str.strip().replace({"Harriston":"Other","Own with Mortgage": "Own", "Parent": "Family", "Reserve Housing": "Other", "Own Free & Clear": "Own", "Own Mobile Home": "Own", "With Parents": "Family"})

**Employment Type**

In [18]:
df['employmentType'] = df['employmentType'].str.strip().replace({"FT": "Full-Time", "PTR": "Part-Time", "SE": "Self-Employed", "SEAS": "Self-Employed", "RET": "Other", "PTC": "Part-Time", "FTP": "Full-Time", "SEAW": "Self-Employed", "PF": "Part-Time", 
                                                           "LT": "Other", "RT": "Other", "SA": "Other", "DI": "Other", "PP": "Other", "PN": "Other", "CN": "Contract", "TF": "Other", "NE": "Other","TP": "Other", "CA": "Other", "": "Other"})

**Booleans**

In [19]:
# not sure why I can link this into a single line, but kept getting errors
df['address.isCurrentResidence'] = df['address.isCurrentResidence'].str.strip()
df['address.isCurrentResidence'] = df['address.isCurrentResidence'].str.lower()
df['address.isCurrentResidence'] = df['address.isCurrentResidence'].replace({'yes': True})
df['address.isCurrentResidence'] = df['address.isCurrentResidence'].fillna(False)

df['creditBureauAvailable'] = df['creditBureauAvailable'].str.strip()
df['creditBureauAvailable'] = df['creditBureauAvailable'].str.lower()
df['creditBureauAvailable'] = df['creditBureauAvailable'].replace({'yes': True})
df['creditBureauAvailable'] = df['creditBureauAvailable'].fillna(False)

df['isCurrentlyEmployed'] = df['isCurrentlyEmployed'].str.strip()
df['isCurrentlyEmployed'] = df['isCurrentlyEmployed'].str.lower()
df['isCurrentlyEmployed'] = df['isCurrentlyEmployed'].replace({'yes': True})
df['isCurrentlyEmployed'] = df['isCurrentlyEmployed'].fillna(False)

df['bankingDataAvailable'] = df['bankingDataAvailable'].str.strip()
df['bankingDataAvailable'] = df['bankingDataAvailable'].str.lower()
df['bankingDataAvailable'] = df['bankingDataAvailable'].replace({'yes': True})
df['bankingDataAvailable'] = df['bankingDataAvailable'].fillna(False)

**Integers**

In [20]:
#remove commas and other fields and ensure just integer string.  Missing go to -1
df['totalNumberOfPayments'] = pd.to_numeric(df['totalNumberOfPayments'], errors='coerce').round().astype('Int64').fillna(-1).astype(str)

df['employmentMonthCount'] = pd.to_numeric(df['employmentMonthCount'], errors='coerce').round().astype('Int64').fillna(-1).astype(str)

df['termMonth'] = pd.to_numeric(df['termMonth'], errors='coerce').round().astype('Int64').fillna(-1).astype(str)

df['paymentsPerYear'] = df['paymentsPerYear'].str.replace('\.00', '')
df['paymentsPerYear'] = pd.to_numeric(df['paymentsPerYear'], errors='coerce').round().astype('Int64').fillna(-1).astype(str)

df['vehicle.odometer'] = df['vehicle.odometer'].str.replace('\.00', '')
df['vehicle.odometer'] = df['vehicle.odometer'].str.replace(r'\D', '')
df['vehicle.odometer'] = pd.to_numeric(df['vehicle.odometer'], errors='coerce').round().astype('Int64').fillna(-1).astype(str)

### Final, cleaned dataframe ...  
**AFTER** all passes in steps 1.3 / 1.4 are good  

In [22]:
print(data.shape)
data.sample(5)

(4564, 78)


,firstName,middleName,lastName,secondLastName,aliases,address.streetAddress,address.city,address.state,address.country,address.postalCode,address.residenceStatus,address.monthlyHousingCost,address.isCurrentResidence,address.monthsAtResidence,phone.mobile,phone.home,phone.work,phone.other,email.primary,email.other,birthday,gender,maritalStatusCd,socialURIs,identifier.type,identifier.value,employer.name,employer.address.streetAddress,employer.address.city,employer.address.state,employer.address.country,employer.address.postalCode,employer.phone,industry,jobTitle,isCurrentlyEmployed,monthlyIncomeNet,monthlyIncomeGross,currentEmployment.paymentsPerYear,employmentMonthCount,employmentType,vehicle.VIN,vehicle.appraisedValue,vehicle.bodyStyle,vehicle.condition,vehicle.make,vehicle.model,vehicle.odometer.value,vehicle.odometer.units,vehicle.trim,vehicle.value,vehicle.year,tradeInAllowance,dealer.address.streetAddress,dealer.address.city,dealer.address.state,dealer.address.country,dealer.address.postalCode,dealer.name,dealer.type,dateOriginated,clientCustomerId,clientLoanReferenceId,principalAmount,annualInterestRate,paymentAmount,paymentsPerYear,totalNumberOfPayments,feeAmount,serviceCharge,interestRate,termMonth,downPayment,jurisdiction.country,jurisdiction.state,custom,creditBureauAvailable,bankingDataAvailable
1288,MARY,J,HEBERT,None,None,,Saint-Charles,NB,CAN,E4W4K7,Other,None,True,<NA>,2896866149,None,None,,None,None,None,F,S,None,,11622179700,ODSP,,Saint-Charles,NB,CAN,E4W4K7,2896866149,Other,ODSP,False,None,None,12,60,Full-Time,,None,None,Used,Chevrolet,Sonic,151927,,LT,None,2012,None,None,Ajax,ON,CAN,None,2396888 Ontario Inc,None,None,33333,DT18476729CRC02,None,None,183.07,12,60,None,None,None,60,300,CAN,ON,"[{""coAppID"":""34034"",""Warehouse_APP_ID"":""29171""...",True,False
2625,TROY,K,MACNIEL,None,None,,Trenton,ON,CAN,K8V2R3,Rent,None,True,<NA>,6132434757,None,None,,None,None,None,M,M,None,,49703080900,GREAT CANADIAN OIL CHANGE,,Trenton,ON,CAN,K8V0B7,6133942585,Professional,LUBE TECHNICIAN,False,None,None,12,24,Full-Time,1G1PE5SB8D7260048,None,None,Used,Chevrolet,Cruze,114561,,LT,None,2013,None,None,Scarborough,ON,CAN,None,Automotive Wholesale Inc.,None,None,71612,DT21841178CRC02,None,34.9,437.97,12,60,None,None,34.9,60,500,CAN,ON,"[{""coAppID"":""71613"",""Warehouse_APP_ID"":""63283""...",True,False
1440,Kenneth,J,Macdonald,None,None,,Welland,ON,CAN,L3C2M4,Rent,None,True,<NA>,2896732117,None,None,,None,None,None,M,M,None,,49821855100,Employers overload,,Etobicoke,ON,CAN,M9B1A7,4162332634,Service,compounder,False,None,None,26,6,Full-Time,KL8CD6S9XFC708174,None,None,Used,Chevrolet,Spark,69668,,1LT,None,2015,None,None,,None,CAN,None,,None,None,38171,DT18921843CRC02,None,29.9,218.61,26,130,None,None,29.9,60,50,CAN,,"[{""coAppID"":""38172"",""Warehouse_APP_ID"":""33459""...",True,False
3295,YOGAHAJAH,,NAGARUBAN,None,None,,Scarborough,ON,CAN,M1K1S2,Family,None,True,<NA>,4165231939,None,None,4165231939,None,None,None,M,M,None,,52275249200,SP IMPORTERS GROCERY,,Scarborough,ON,CAN,,4162613881,Sales,ACCOUNTING ASSISTANT,False,None,None,26,24,Full-Time,3G1BE5SM9HS507628,None,None,Used,Chevrolet,Cruze,76608,,LT,None,2017,None,None,Richmond Hill,ON,CAN,None,RICHMOND HILL SUBARU,None,None,92803,DT23606447CRC02,None,None,115.04,26,156,None,None,None,72,None,CAN,ON,"[{""coAppID"":""92804"",""Warehouse_APP_ID"":""82254""...",True,False
3507,Teresa,L,Rose,None,None,,Tara,ON,CAN,N0H2N0,Rent,None,True,<NA>,2266682358,None,None,2266682358,None,None,None,M,M,None,,50944690200,Retired,,Tara,ON,CAN,N0H2N0,2266682358,Self-Employed,Homemaker,False,None,None,12,48,Full-Time,3N1BC1CP9CL368461,None,None,Used,Nissan,Versa,163642,,SL,None,2012,None,None,Cambridge,ON,CAN,None,2615497 Ontario Limited,None,None,98964,DT24092771CRC02,None,34.9,466.03,12,60,None,None,34.9,60,500,CAN,ON,"[{""coAppID"":""98965"",""Warehouse_APP_ID"":""87801""...",True,False


### 1.5. Saving the resulted dataframe to s3

In [23]:
destinationBucket = "trustsciencespark"
destinationKey = "axisautofinance-wh/rawData/WH-3151-Axis-NonFunded-CoApp.csv"

shared_lib.save_to_s3(destinationBucket, destinationKey, data.to_csv(index=False))

INFO: File sucessfully uploaded to S3


s3.Object(bucket_name='trustsciencespark', key='axisautofinance-wh/rawData/WH-3151-Axis-NonFunded-CoApp.csv')

## 2. Validation API (run a deeper validation on the SchemaV2)  

**NOTE** VALIDATION MAY OVERWRITE - saves to Jira name. 
- because the NonFunded MAINAPP were done with same jira as COAPPS (WH-3151), check versions!

In [24]:
path_csv = "axisautofinance-wh/rawData/WH-3151-Axis-NonFunded-CoApp.csv"
bucket_name = "trustsciencespark"

In [25]:
validation_api = "https://u5neqcth6i.execute-api.us-west-2.amazonaws.com/dev/poc/automation/validation"

In [26]:
validation_request = {
    "messageType": "CsvValidation",
    "messageContext": {
        "clientID": "axisautofinance-wh",
        "userId": "rob",
        "source": "WH-3151",
        "env": "dev",
        "custom": {
            "enablePrepareLionRequest": False,
            "enableSubmitBulk": True
        }
    },
    "messageData": {
        "dataPath": 
        [
            path_csv
        ],
        "bucketName": bucket_name
    }
}

In [27]:
res = requests.post(validation_api, json=validation_request)
print(res.status_code)
print(res.text)

200
{"status": "Success", "message": "Request of CSV validation was submitted successfully, and the validation report is saved into s3", "reportUrl": "https://s3.console.aws.amazon.com/s3/buckets/trustsciencespark/axisautofinance-wh/WH-3151/validationResult/?region=us-west-2&tab=overview"}


## Go to 'reportUrl' link and check the validation results

** MANUALLY RENAMED VALIDATION BECAUSE DECLINES USES SAME JIRA **  
- added prefix: WH-3151-B-   

### Generate Profile  
- shows what the uploaded data look like ...   
- attach to jira

In [28]:
resultsFolder = './results'
if not os.path.exists(resultsFolder):
    os.makedirs(resultsFolder)
resultsFolder = resultsFolder + '/'

In [29]:
reportName = "WH-3151-B-ProfileReport"
profile = data.profile_report(title=reportName, minimal=True)
profile.to_file(output_file=resultsFolder + reportName + ".html")

# DONE